In [ ]:
import numpy as np
import pandas as pd
from resolve_path import ajuste_path

In [ ]:
path = "data/util/"
path = ajuste_path(path)
df = pd.read_csv(path + "dataset_treinamento.csv")

In [ ]:
print("Quantidade de linhas antes de retirar locais com 0 acidentes", len(df))

In [ ]:
df.info()
df_copy = df.copy()

INSERINDO HH EM LI QUE NÃO TEM HH

In [ ]:
def retorna_soma_HH_por_locais(df, local1, local2, ano, mes):
    # Filtrar as linhas correspondentes
    df_filtrado = df[(df["Ano"] == ano) & (df["Mes"] == mes) & (
        df["Local de instalação"].str.split("-").str[-1].isin([local1, local2]))]

    # Sumarizar as colunas que começam com "HH"
    soma_HH = df_filtrado.filter(regex='^HH').sum() / 2

    return soma_HH.to_dict()


def extrai_locais_de_linhas_de_transmissao(lt):
    partes = lt.split("-")
    locais = partes[-1].split("/")
    return locais[0], locais[1]


# Condição otimizada
condicao = "`Quantidade de Acidentes` > 0 and `Local de instalação`.str.startswith('S-L') and `HH total` == 0"

count = 0

for index, linha in df.query(condicao).iterrows():
    count += 1
    local1, local2 = extrai_locais_de_linhas_de_transmissao(
        linha["Local de instalação"])
    soma_HH = retorna_soma_HH_por_locais(
        df, local1, local2, linha["Ano"], linha["Mes"])

    # Atualizar as colunas no DataFrame original
    for col, valor in soma_HH.items():
        df.loc[index, col] = valor

print("Quantidade de linhas alteradas: ", count)
print("(Depois) Quantidade de linhas com acidentes e hh igual a 0: ",
      df.query("`Quantidade de Acidentes` > 0 and `HH total` == 0").shape[0])

CRIAÇÃO DE TABELA DE LOCAIS E ACIDENTES

In [ ]:
df_nv = pd.DataFrame(columns=['Local', 'Acidentes'])

for index, row in df.iterrows():
    local = row['Local de instalação']
    acidentes = row['Quantidade de Acidentes']
    if local in df_nv['Local'].values:
        df_nv.loc[(df_nv["Local"] == local), "Acidentes",] += acidentes
    else:
        df_nv = pd.concat(
            [
                df_nv,
                pd.DataFrame(
                    {
                        "Local": [local],
                        "Acidentes": [acidentes]
                    }
                ),
            ]
        )


pd.options.display.max_rows = 500
df_nv

SEPARANDO APENAS LOCAIS COM 0 ACIDENTES

In [ ]:
zero_acidentes = df_nv.loc[df_nv['Acidentes'] == 0]
zero_acidentes

TIRANDO LOCAIS QUE NAO TEM ACIDENTES DO DATASET DE ENTRADA

In [ ]:
for indx, row in df.iterrows():
    local = row['Local de instalação']
    if local in zero_acidentes['Local'].values:
        df = df.drop(index=indx)

In [ ]:
print("Quantidade de linhas depois de retirar locais com 0 acidentes", len(df))